# Decathlon Turnover - Data Exploration

## Get Started

In [101]:
# import packages
import datetime

import pandas as pd

from statsmodels.tsa.seasonal import seasonal_decompose

In [102]:
## env variables
raw_data_folder = "../data/raw"
processed_data_folder = "../data/processed"

## Data setup

We distinguish three data files:

- `train.csv`: dataset containing stores/departments related features and weekly turnover data for training purpose. The training dataset contains the _turnover_ variable to predict.
- `test.csv`: testing dataset has the same structure as `train.csv` without the turnover variable to predict.
- `bu_feat.csv`: contains more stores' related features like geolocalisation, region, postcode... etc.

The scope of our data concerns **4 departments** and **322 stores**.

In [103]:
# loading datasets
df_train = pd.read_csv(f"{raw_data_folder}/train.csv", encoding="utf-8")
df_test = pd.read_csv(f"{raw_data_folder}/test.csv", encoding="utf-8")
df_features = pd.read_csv(f"{raw_data_folder}/bu_feat.csv", encoding="utf-8")

# data description
print("Training dataset contains {} rows and {} columns.".format(*df_train.shape))
print("Testing dataset contains {} rows and {} columns.".format(*df_test.shape))
print("Features dataset contains {} rows and {} columns.".format(*df_features.shape))

Training dataset contains 277719 rows and 4 columns.
Testing dataset contains 10136 rows and 3 columns.
Features dataset contains 326 rows and 6 columns.


In [104]:
# checking training dataset
df_train.head()

,day_id,but_num_business_unit,dpt_num_department,turnover
0,2017-09-30,64,127,580.308443
1,2017-09-30,119,127,1512.995918
2,2017-09-30,4,88,668.593556
3,2017-09-30,425,127,0.000000
4,2017-09-30,513,73,0.000000


In [105]:
# checking test dataset
df_test.head()

,day_id,but_num_business_unit,dpt_num_department
0,2017-11-25,95,73
1,2017-11-25,4,117
2,2017-11-25,113,127
3,2017-11-25,93,117
4,2017-11-25,66,127


In [106]:
# checking store's related features
df_features.head()

,but_num_business_unit,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr
0,1609,80100,50.096306,1.867220,69,4
1,1995,95700,48.991087,2.520903,3,6
2,1800,73210,45.552933,6.644736,51,4
3,238,47550,44.175370,0.638869,134,10
4,201,13546,43.508418,5.406423,71,10


## Features Engineering

First of all, we start by adding new features from the existing ones to answer the required questions and also building a solid forecasting model.

Temporal features are very important, it allows us to simplify the data exploration, for example turnover's insights related to a:
  - year,
  - month,
  - week of year,
  - day of week,
  - quarter...

In [107]:
# generate new temporal features from date
df_train["year"] = pd.DatetimeIndex(df_train["day_id"]).year
df_train["month"] = pd.DatetimeIndex(df_train["day_id"]).month
df_train["week_of_year"] = pd.DatetimeIndex(df_train["day_id"]).weekofyear
df_train["day_of_week"] = pd.DatetimeIndex(df_train["day_id"]).dayofweek
df_train["quarter"] = pd.DatetimeIndex(df_train["day_id"]).quarter

/var/folders/p9/xnwkddgd6378dzdjcns4xlxw0000gn/T/ipykernel_74151/2967261162.py:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df_train["week_of_year"] = pd.DatetimeIndex(df_train["day_id"]).weekofyear


To have the complete training dataset, we must merge the store's related features to the main dataset.

In [108]:
# merging store features and training dataset
df_train_c = df_train.merge(
    df_features, left_on="but_num_business_unit", right_on="but_num_business_unit", how="left"
)
df_train_c.head()

,day_id,but_num_business_unit,dpt_num_department,turnover,year,month,week_of_year,day_of_week,quarter,but_postcode,but_latitude,but_longitude,but_region_idr_region,zod_idr_zone_dgr
0,2017-09-30,64,127,580.308443,2017,9,39,5,3,16400,45.625172,0.111939,70,10
1,2017-09-30,119,127,1512.995918,2017,9,39,5,3,74100,46.195037,6.254448,51,4
2,2017-09-30,4,88,668.593556,2017,9,39,5,3,6600,43.600994,7.078160,55,10
3,2017-09-30,425,127,0.000000,2017,9,39,5,3,59000,50.617921,3.084186,33,3
4,2017-09-30,513,73,0.000000,2017,9,39,5,3,33610,44.717366,-0.733429,33,3


## FAQ

- Which department made the highest turnover in 2016?
- What are the top 5 week numbers (1 to 53) for department 88 in 2015 in terms of turnover over all stores?
- What was the top performer store in 2014?
- Based on sales can you guess what kind of sport represents department 73?
- Based on sales can you guess what kind of sport represents department 117?
- What other insights can you draw from the data? Provide plots and figures if needed. (Optional)

In [109]:
# this code list and sort the departments by turnover earnings in 2016
# we see that department number 127 produced the highest turnover that year
tmp = df_train_c[df_train_c["year"] == 2016][["dpt_num_department", "turnover"]]
tmp.groupby(["dpt_num_department"]).sum().sort_values(by=["turnover"], ascending=False)

,turnover
dpt_num_department,
127,3.378005e+07
117,6.535299e+06
88,5.696284e+06
73,1.974491e+06


In [110]:
tmp = df_train_c[(df_train_c["year"] == 2015) & (df_train_c["dpt_num_department"] == 88)]
tmp = tmp[["week_of_year", "turnover"]]
tmp.groupby(["week_of_year"]).sum().sort_values(by=["turnover"], ascending=False).head()

,turnover
week_of_year,
27,141679.864768
37,139674.400588
36,137825.642436
38,137232.460861
28,125835.541415


In [111]:
tmp = df_train_c[df_train_c["year"] == 2014]
tmp = tmp[["but_num_business_unit", "turnover"]]
tmp.groupby(["but_num_business_unit"]).sum().sort_values(by=["turnover"], ascending=False).head()

,turnover
but_num_business_unit,
121,327806.356464
17,311496.294164
118,301938.962571
189,296147.653207
100,285379.195910


In [112]:
tmp = df_train_c[df_train_c["dpt_num_department"] == 73]
tmp = tmp[["month", "turnover"]]
tmp.groupby(["month"]).sum().sort_values(by=["turnover"], ascending=False)

,turnover
month,
7,778792.450601
8,616001.610128
6,574713.057621
5,572157.389881
4,529467.772647
9,451552.783590
10,406385.546014
1,347841.130309
11,312737.416412


## Data persistence

In [113]:
# save the processed datasets
df_train_c.to_csv("{}/train.csv".format(processed_data_folder), sep=";", index=False)
df_test.to_csv("{}/test.csv".format(processed_data_folder), sep=";", index=False)